# SocketsServerJava

In [ ]:
System.out.println("Hola Mundo");

## Sockets básicos en java

Ejecutar cada una de las subsecciones (server, client) siguientes en dos notebooks diferentes

### Server

In [ ]:
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.ServerSocket;
import java.net.Socket;

public class Servidor {
    public static void main() {
        int puerto = 8080;

        try (ServerSocket serverSocket = new ServerSocket(puerto)) {
            System.out.println("Servidor escuchando en el puerto: " + puerto);

            while (true) {
                try (Socket socket = serverSocket.accept()) {
                    BufferedReader input = new BufferedReader(new InputStreamReader(socket.getInputStream()));
                    PrintWriter output = new PrintWriter(socket.getOutputStream(), true);

                    String mensaje = input.readLine();
                    System.out.println("Mensaje recibido del cliente: " + mensaje);

                    String respuesta = "Mensaje recibido: " + mensaje;
                    output.println(respuesta);
                }
            }
        } catch (Exception e) {
            System.out.println("Error en el servidor: " + e.getMessage());
        }
    }
}

In [ ]:
Servidor.main();

### Client

In [ ]:
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.Socket;

public class Cliente {
    public static void main() {
        String host = "localhost";
        int puerto = 8080;

        try (Socket socket = new Socket(host, puerto)) {
            BufferedReader input = new BufferedReader(new InputStreamReader(socket.getInputStream()));
            PrintWriter output = new PrintWriter(socket.getOutputStream(), true);

            String mensaje = "Todo Bien?";
            output.println(mensaje);

            String respuesta = input.readLine();
            System.out.println("Respuesta del servidor: " + respuesta);
        } catch (Exception e) {
            System.out.println("Error en el cliente: " + e.getMessage());
        }
    }
}

In [ ]:
Cliente.main();

## Servidor Avanzado

In [ ]:
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.ServerSocket;
import java.net.Socket;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
import java.util.concurrent.Semaphore;

public class ServidorMultihilos {

    public static void main() {
        int puerto = 8070;
        int maxClientes = 3;
        ExecutorService executor = Executors.newFixedThreadPool(maxClientes);
        Semaphore semaforo = new Semaphore(maxClientes);

        try (ServerSocket serverSocket = new ServerSocket(puerto)) {
            System.out.println("Iniciando servidor en el puerto: " + puerto);

            while (true) {
                System.out.println("Esperando una conexión...");
                Socket socket = serverSocket.accept();

                semaforo.acquire();
                Runnable clientHandler = () -> {
                    procesarCliente(socket, semaforo);
                };
                executor.execute(clientHandler);
            }
        } catch (Exception e) {
            System.out.println("Error en el servidor: " + e.getMessage());
        }
    }

    public static void procesarCliente(Socket socket, Semaphore semaforo) {
        try {
            System.out.println("Conexión desde " + socket.getRemoteSocketAddress());

            BufferedReader input = new BufferedReader(new InputStreamReader(socket.getInputStream()));
            PrintWriter output = new PrintWriter(socket.getOutputStream(), true);

            String mensajeBienvenida = "Bienvenido al servidor! ¿Cómo puedo ayudarte?\n";
            output.println(mensajeBienvenida);

            String mensaje;
            while ((mensaje = input.readLine()) != null) {
                System.out.println("Mensaje recibido desde " + socket.getRemoteSocketAddress() + ": " + mensaje);

                String respuesta = "Tu mensaje \"" + mensaje + "\" ha sido recibido. Gracias por contactarnos!\n";
                output.println(respuesta);

                Thread.sleep(1000);
            }

            socket.close();
            System.out.println("Conexión cerrada con " + socket.getRemoteSocketAddress());

        } catch (Exception e) {
            System.out.println("Error en el procesamiento del cliente: " + e.getMessage());
        } finally {
            semaforo.release();
        }
    }
}

In [ ]:
ServidorMultihilos.main();